In [8]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from imutils import contours
import imutils
import glob
import os
import pandas as pd
import decimal
import copy

In [9]:
video_path1 = "firstvid.mp4"
video_path2 = "secondvid.mp4"

In [10]:
print("Enter 1 for video 1, and 2 for video 2")
vid = int(input())

if vid == 1:
    cap = cv2.VideoCapture(video_path1)
    [start_x, start_y, end_x, end_y] = [200, 200, 1450, 900]
    start_frame = 90
    if cap.isOpened() is False:
        print("Error opening video stream or file")
    print(f"Total frame count for {video_path1} is: {cap.get(cv2.CAP_PROP_FRAME_COUNT)} and has {cap.get(cv2.CAP_PROP_FPS)} frames per second")
    print(f"Video size is {int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))} X {int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))}")
else:   
    cap = cv2.VideoCapture(video_path2)
    [start_x, start_y, end_x, end_y] = [500, 300, 1600, 800]
    start_frame = 0
    if cap.isOpened() is False:
        print("Error opening video stream or file")
    print(f"Total frame count for {video_path2} is: {cap.get(cv2.CAP_PROP_FRAME_COUNT)} and has {cap.get(cv2.CAP_PROP_FPS)} frames per second")
    print(f"Video size is {int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))} X {int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))}")

Enter 1 for video 1, and 2 for video 2
1
Total frame count for firstvid.mp4 is: 313.0 and has 30.0 frames per second
Video size is 1920 X 1080


In [11]:
def display(name, image):
    cv2.imshow(name, image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [12]:
df = pd.DataFrame(columns=['Frame_Index', 'Area', 'Mean', 'Radius', 'Rotated_Box'])

# Optimizing Splash Pool

In [13]:
%%time

frame_index = 0
frame = []

while cap.isOpened():
    if frame_index % 100 == 0:
        print(f"Processing frame: {frame_index}")
    ret_val, image = cap.read()
    if image is None:
        break
    frame_index = frame_index + 1
    if frame_index < start_frame:
        continue
        
    
    cropped = image[start_y:end_y, start_x:end_x]
    frame.append(cropped)
    copy1 = copy.deepcopy(cropped)
    copy2 = copy.deepcopy(cropped)

    
    gray = cv2.cvtColor(cropped, cv2.COLOR_BGR2GRAY)
    frame.append(gray)
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    frame.append(blur)
    thresh = cv2.threshold(blur, 200, 255, cv2.THRESH_BINARY)[1]
    frame.append(thresh)
    erode = cv2.erode(thresh, None, iterations = 1)
    frame.append(erode)
    dilate = cv2.dilate(erode, None, iterations = 1)
    frame.append(dilate)
    cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    #cv2.drawContours(cropped, cnts[0], -1, (0,255,0), 3)
    #frame.append(cropped)
    
    
    cnts = imutils.grab_contours(cnts)

    if len(cnts) != 0:
        sorted_contours = sorted(cnts, key=cv2.contourArea, reverse=True)
        
        # Get top 2 contours (2nd largest if available) and combine their points to get encompassing rectangle.
        largest_contour = sorted_contours[0]
        top_contour_points = largest_contour
        cv2.drawContours(copy1, top_contour_points, -1, (0,255,0), 3)
        
        # Get area of melt pool (largest + second largest if available).
        area = cv2.contourArea(sorted_contours[0])
        if len(cnts) >= 2:
            second_largest_contour = sorted_contours[1]
            top_contour_points = np.concatenate((largest_contour,second_largest_contour))
            cv2.drawContours(copy1, sorted_contours[1], -1, (0,255,0), 3)
            #display("Second contour", copy2)
            area = area + cv2.contourArea(second_largest_contour)
        # Get average intensity of melt pool.
        contour_mask = np.zeros(gray.shape, np.uint8)
        cv2.drawContours(contour_mask, top_contour_points, -1, 255, -1) # Draw filled contours.
        frame.append(contour_mask)
        mean = cv2.mean(gray, mask=contour_mask)

        # Get radius of minimum enclosing circle of contour(s).
        ((cX, cY), radius) = cv2.minEnclosingCircle(top_contour_points)
        
        # Find rotated rectangle of encompassing contours. 
        rotated_rectangle = cv2.minAreaRect(top_contour_points)
        rotated_box = cv2.boxPoints(rotated_rectangle)
        rotated_box = np.int0(rotated_box)

        # Draw the rotated rectangle.
        #cv2.drawContours(copy1,[rotated_box],0,(0,0,255),2)
        frame.append(copy1)
        
        df = df.append({'Frame_Index': f'{frame_index:07}', 'Area': area, 'Mean': mean[0], 'Radius': radius, 'Rotated_Box': rotated_box}, ignore_index=True)

    else:
        df = df.append({'Frame_Index': f'{frame_index:07}', 'Area': 0, 'Mean': 0, 'Radius': 0}, ignore_index=True)
    
    
    
        

fps = int(cap.get(cv2.CAP_PROP_FPS))
height, width, channels = frame[0].shape
size = (width, height)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
writer = cv2.VideoWriter("outputvid.mp4", fourcc, fps, (width, height))

for i in range(len(frame)):
    writer.write(frame[i])
    
    
writer.release()

cap.release()
cv2.destroyAllWindows()

Processing frame: 0


<timed exec>:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:73: FutureWarning: The frame.append method is deprecated and will be removed from panda

Processing frame: 100


<timed exec>:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:73: FutureWarning: The frame.append method is deprecated and will be removed from panda

<timed exec>:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:73: FutureWarning: The frame.append method is deprecated and will be removed from panda

...

<timed exec>:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:73: FutureWarning: The frame.append method is deprecated and will be removed from panda

<timed exec>:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:73: FutureWarning: The frame.append method is deprecated and will be removed from panda

Processing frame: 300


<timed exec>:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


CPU times: total: 10.7 s
Wall time: 6.64 s


In [118]:
df.head()

,Frame_Index,Area,Mean,Radius,Rotated_Box
0,0000090,6936.5,215.306977,84.353661,"[[722, 519], [846, 445], [919, 569], [795, 642]]"
1,0000091,6246.0,213.234043,68.679382,"[[559, 533], [656, 494], [703, 611], [606, 650]]"
2,0000092,6245.0,213.346505,68.679382,"[[559, 533], [656, 494], [703, 611], [606, 650]]"
3,0000093,6146.0,219.362869,89.898659,"[[658, 580], [702, 474], [861, 540], [817, 646]]"
4,0000094,4821.0,220.224490,73.321327,"[[654, 599], [693, 464], [779, 489], [741, 624]]"


In [119]:
df.to_csv("pool_data_from_video1.csv",index=False)